In [1]:
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_lines(day):
    with open(f'data/2021/{day}.txt', 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

def read_ints(day):
    lines = read_lines(day)
    return list(map(int, lines))

In [3]:
def day1():
    vals = read_ints(1)
    part1 = sum(np.diff(vals) > 0)
    
    windowed = np.convolve(vals, np.ones(3), mode='valid')
    part2 = sum(diff(windowed) > 0)
    return (part1, part2)

In [4]:
def day2():
    lines = read_lines(2)
    pairs = [line.split() for line in lines]
    pairs = [(d, int(v)) for (d,v) in pairs]
    
    x = 0; z = 0
    for (d, v) in pairs:
        if d == 'forward':
            x = x + v
        elif d == 'down':
            z = z + v
        elif d == 'up':
            z = z - v
    p1 = x * z
            
    x = 0; z = 0; aim = 0
    for (d, v) in pairs:
        if d == 'forward':
            x = x + v
            z = z + aim * v
        elif d == 'down':
            aim = aim + v
        elif d == 'up':
            aim = aim - v
    p2 = x * z
    
    return (p1, p2)

In [5]:
run = [day1, day2]

for day in run:
    (p1, p2) = day()
    print(day.__name__)
    print('part 1: ', p1)
    print('part 2: ', p2)

day1
part 1:  1521
part 2:  1543
day2
part 1:  1451208
part 2:  1620141160
